In [2]:
!pip install internetarchive html2text
!sudo apt-get install parallel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 3.4 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=49478d8c90ac03de11a2305e7d6680e5d4c22a64c9b62cab1157ef278878fc9d
  Stored in directory: /root/.cache/pip/wheels/23/58/7c/d9c8c4d924a1ac2b621add1b2c1d30b639629a33cfdfde6a45
Successfully built html2text


In [1]:
!html2text --version
!parallel --version
!nproc

Unrecognized command line option "--version", try "-help".
GNU parallel 20231122
Copyright (C) 2007-2023 Ole Tange, http://ole.tange.dk and Free Software
Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
GNU parallel comes with no warranty.

Web site: https://www.gnu.org/software/parallel

When using programs that use GNU Parallel to process data for publication
please cite as described in the manpage.
16


In [29]:
%%time
# ~10 dakika
from internetarchive import download

download('5000-epub-kitap', verbose=True, glob_pattern='*Agatha*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Tolstoy*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Tolkien*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Asimov*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Adıvar*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Dostoyevski*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Diyanet*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Herbert*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Kafka*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Kollektif*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Komisyon*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Tolstoy*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Reşat Nuri*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Özdenören*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Seyfettin*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Kısakürek*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*Köksal*html')
download('5000-epub-kitap', verbose=True, glob_pattern='*İnalcık*html')

5000-epub-kitap:
 skipping 5000-epub-kitap/5000 Epub kitap/A Harfi/Agatha Christie/Agatha Christie - 16.50 Treni_hocr.html, file already exists based on length and date.
 skipping 5000-epub-kitap/5000 Epub kitap/A Harfi/Agatha Christie/Agatha Christie - Arsenik_hocr.html, file already exists based on length and date.
 skipping 5000-epub-kitap/5000 Epub kitap/A Harfi/Agatha Christie/Agatha Christie - Beklenmeyen Misafir_hocr.html, file already exists based on length and date.
 skipping 5000-epub-kitap/5000 Epub kitap/A Harfi/Agatha Christie/Agatha Christie - Beş Küçük Domuz_hocr.html, file already exists based on length and date.
 skipping 5000-epub-kitap/5000 Epub kitap/A Harfi/Agatha Christie/Agatha Christie - Bir Kadeh Şampanya_hocr.html, file already exists based on length and date.
 skipping 5000-epub-kitap/5000 Epub kitap/A Harfi/Agatha Christie/Agatha Christie - Cesetler Merdiveni_hocr.html, file already exists based on length and date.
 skipping 5000-epub-kitap/5000 Epub kitap/A

CPU times: user 46.2 s, sys: 15.4 s, total: 1min 1s
Wall time: 7min 53s


[]

In [47]:
!du -sh 5000-epub-kitap

7.1G	5000-epub-kitap


In [30]:
%%time
#33 dakika
!find -name '*.html' -print0 | parallel -0 -j2 html2text {} ">" {.}.md

CPU times: user 11.5 s, sys: 1.78 s, total: 13.2 s
Wall time: 33min 3s


In [31]:
!rm -fr sample_data
!mkdir md
!find . -name '*.md' -exec mv {} md/ \;

mkdir: cannot create directory ‘md’: File exists
mv: './md/Agatha Christie - Arsenik_hocr.md' and 'md/Agatha Christie - Arsenik_hocr.md' are the same file
mv: './md/Agatha Christie - 16.50 Treni_hocr.md' and 'md/Agatha Christie - 16.50 Treni_hocr.md' are the same file
mv: './md/Agatha Christie - Beş Küçük Domuz_hocr.md' and 'md/Agatha Christie - Beş Küçük Domuz_hocr.md' are the same file
mv: './md/Agatha Christie - Beklenmeyen Misafir_hocr.md' and 'md/Agatha Christie - Beklenmeyen Misafir_hocr.md' are the same file


In [48]:
!ls -l md/ | wc -l

301


In [41]:
!mkdir md/test
!mv "md/Hobbit - J.R.R. Tolkien_hocr.md" md/test
!rm "md/J.R.R. Tolkien - Hobbit (Resimli)_hocr.md"

In [3]:
import glob
tokendata = ''
for file in glob.glob("md/*.md"):
    with open(file,'r') as f:
        tokendata += f.read(20000)[10000:]
len(tokendata)

2990000

In [2]:
def merge(tokens,pair,idx):
    data = list(tokens)
    i = 1
    while i < len(data):
        p = (data[i-1],data[i])
        if p == pair:
            data[i-1] = idx
            data.pop(i)
        i += 1
    return data

def merge_fast(tokens, pair, idx):
    data = list(tokens)
    result = []
    i = 0
    n = len(data)

    while i < n:
        if i < n - 1 and (data[i], data[i + 1]) == pair:
            result.append(idx)
            i += 2  # Skip the next token since we've merged it
        else:
            result.append(data[i])
            i += 1

    return result

def mostfrequent(data):
    stats = {}
    count = 0
    mpair = (0,0)
    for i in range(1,len(data)):
        pair = (data[i-1],data[i])
        stats[pair] = stats.get(pair,0) + 1
        if count < stats[pair]:
            count = stats[pair]
            mpair = pair
    return mpair
test = [110,111,112,111,112]
mostfrequent(test), merge_fast(test,(111,112),0)

((111, 112), [110, 0, 0])

In [4]:
encoded = tokendata.encode("utf-8")
orglen = len(encoded)
print(f"Data length:{len(encoded)}")
merges = {}

Data length:3289048


In [5]:
%%time
from pickle import dump, load
# ~30 dakika
charmap = { i:bytes([i]) for i in range(256) }
while len(merges) < 4096-256-10:
    mf = mostfrequent(encoded)
    merges[mf] = 256+len(merges)
    charmap[merges[mf]] = charmap[mf[0]] + charmap[mf[1]]
    try:
        t = charmap[merges[mf]].decode('utf-8')
    except Exception as e:
        t = str(list(charmap[merges[mf]]))
    encoded = merge_fast(encoded,mf,merges[mf])
    print(f"{merges[mf]:4}:({mf[0]:4},{mf[1]:4}) = [{t.ljust(10)}] {(len(encoded)/orglen)*100:.2f}%")
    i = 1


with open(f'merges_{len(merges)}.pkl','wb') as f:
    dump(merges,f)

 256:( 196, 177) = [ı         ] 96.49%
 257:( 110,  32) = [n         ] 95.01%
 258:( 195, 188) = [ü         ] 93.63%
 259:( 101,  32) = [e         ] 92.36%
 260:(  97, 114) = [ar        ] 91.11%
 261:( 197, 159) = [ş         ] 89.88%
 262:(  97,  32) = [a         ] 88.73%
 263:( 101, 114) = [er        ] 87.65%
 264:( 105,  32) = [i         ] 86.68%
 265:( 105, 114) = [ir        ] 85.82%
 266:(  44,  32) = [,         ] 84.98%
 267:(  46,  32) = [.         ] 84.19%
 268:( 105, 110) = [in        ] 83.42%
 269:( 195, 167) = [ç         ] 82.67%
 270:( 196, 159) = [ğ         ] 81.93%
 271:(  97, 110) = [an        ] 81.20%
 272:( 256, 110) = [ın        ] 80.60%
 273:( 256,  32) = [ı         ] 80.01%
 274:(  97, 107) = [ak        ] 79.45%
 275:( 105, 108) = [il        ] 78.90%
 276:( 195, 182) = [ö         ] 78.35%
 277:( 100, 101) = [de        ] 77.82%
 278:( 108, 260) = [lar       ] 77.31%
 279:(  98, 265) = [bir       ] 76.82%
 280:( 101, 110) = [en        ] 76.35%
 281:(  97, 108) = [al   

NameError: name 'decode' is not defined

In [11]:
def encode_fast(string):
    encoded = string.encode("utf-8")
    for m in merges:
        encoded = merge_fast(encoded,m,merges[m])
    return encoded

def decode(encoded):
    charmap = { i:bytes([i]) for i in range(256) }
    for pair in merges:
        charmap[merges[pair]] = charmap[pair[0]] + charmap[pair[1]]
    uncompressed = b''.join([ charmap[d] for d in encoded ])
    #print(list(uncompressed))
    try:
        decoded = uncompressed.decode('utf-8')
    except Exception as e:
        decoded = list(uncompressed)
    return decoded

encode_fast('Türkiye')

[3955, 101]

In [16]:
with open('merges.txt','w') as f:
    for i,m in enumerate(merges):
        f.write(f"{i+256:4}:({m[0]:4},{m[1]:4}):{decode(m)}\n")

In [7]:
from concurrent.futures import ProcessPoolExecutor

def parallel_encode(string, num_chunks=16):
    # Verilen string'i parçalara böl
    chunk_size = len(string) // num_chunks
    chunks = [string[i:i + chunk_size] for i in range(0, len(string), chunk_size)]

    # Paralel bir şekilde encode_fast fonksiyonunu çağır
    with ProcessPoolExecutor() as executor:
        # .map() ile yapınca, parçaların sırası korunuyor
        encoded_chunks = list(executor.map(encode_fast, chunks))

    # Sonuçları birleştir
    final_encoded = []
    for e in encoded_chunks: final_encoded += e
    return final_encoded

In [8]:
%%time
# ~2 saat 15 dakika
import os

for file in glob.glob("md/*.md"):
    with open(file,'r') as f:
        print(file)
        if os.path.exists(f'{file}_tokens.pkl'): continue
        tokens =  parallel_encode(f.read()) + [4086] #endof document
        print(len(tokens))
        with open(f'{file}_tokens.pkl','wb') as f:
            dump(tokens,f)

md/Agatha Christie - Sıfıra Doğru_hocr.md
78204
md/Fyodor Mihailoviç Dostoyevski - Kadın Budalası_hocr.md
80060
md/Isaac Asimov - İmparatorluk #3 Uğursuz Gezegen Galactica_hocr.md
113586
md/Fyodor Mihailoviç Dostoyevski - Budala (İş Bankası)_hocr.md
419582
md/J.R.R. Tolkien - Peri Masalları Üzerine_hocr.md
47738
md/Fyodor Mihailoviç Dostoyevski - Karamazov Kardeşler_hocr.md
536940
md/Isaac Asimov - Robot #3 Şafağın Robotları_hocr.md
183668
md/Necip Fazıl Kısakürek - Doğru Yolun Sapık Kolları_hocr.md
79070
md/Bir Koy Hekimi - Franz Kafka_hocr.md
24750
md/Frank Herbert - Dune #1 Çöl Gezegeni Dune_hocr.md
367307
md/Rasim Özdenören - Yazı, İmge ve Gerçeklik_hocr.md
77337
md/Isaac Asimov - Robot #1 - Ben Robot_hocr.md
113419
md/Lev Nikolayeviç Tolstoy - Savaş ve Barış (Can Yayınları )_hocr.md
927438
md/Fyodor Mihailoviç Dostoyevski - Yeraltından Notlar_hocr.md
66124
md/Reşat Nuri Güntekin - Damga_hocr.md
71218
md/Fyodor Mihailoviç Dostoyevski - Bir Yazarın Günlüğü_hocr.md
749433
md/Isaac As

In [13]:
%%time
import os

for file in glob.glob("md/test/*.md"):
    with open(file,'r') as f:
        print(file)
        if os.path.exists(f'{file}_tokens.pkl'): continue
        tokens =  parallel_encode(f.read()) + [4086] #endof document
        print(len(tokens))
        with open(f'{file}_tokens.pkl','wb') as f:
            dump(tokens,f)

md/test/Hobbit - J.R.R. Tolkien_hocr.md
157337
CPU times: user 15.7 ms, sys: 1.07 s, total: 1.09 s
Wall time: 10.7 s


In [14]:
from pickle import dump, load
import glob
# 157.337 token
valTokens = []
for file in glob.glob("md/test/*.pkl"):
    with open(file,'rb') as f:
        print(file)
        valTokens += load(f)
print(len(valTokens))

with open(f'val.pkl','wb') as f:
    dump(valTokens,f)

md/test/Hobbit - J.R.R. Tolkien_hocr.md_tokens.pkl
157337


In [15]:
#128.314.806
trainTokens = []
for file in glob.glob("md/*.pkl"):
    with open(file,'rb') as f:
        print(file)
        trainTokens += load(f)
print(len(trainTokens))

with open(f'train.pkl','wb') as f:
    dump(trainTokens,f)

md/Diyanet Vakfı İslam Ansiklopedisi - 30 (Mısra)_hocr.md_tokens.pkl
md/Rasim Özdenören - Edebiyat ve Hayat_hocr.md_tokens.pkl
md/Isaac Asimov - Sonsuzluğun Sonu_hocr.md_tokens.pkl
md/Fyodor Mihailoviç Dostoyevski - Budala(Antik)_hocr.md_tokens.pkl
md/Lev Nikolayeviç Tolstoy - Şeytan_hocr.md_tokens.pkl
md/Rasim Özdenören - İpin Ucu_hocr.md_tokens.pkl
md/Brian Herbert & Kevin J. Anderson - Dune Corrin Savaşı_hocr.md_tokens.pkl
md/Diyanet Vakfı İslam Ansiklopedisi - 33 (Nesih)_hocr.md_tokens.pkl
md/Agatha Christie - Hercule Poirot #38 - Hercule Pairot İz Üzerinde_hocr.md_tokens.pkl
md/Reşat Nuri Güntekin - Olağan İşler_hocr.md_tokens.pkl
md/Komisyon - Türkler Cilt 02 İlkçağ_hocr.md_tokens.pkl
md/Komisyon - Türkler Cilt 00 İçindekiler_hocr.md_tokens.pkl
md/Isaac Asimov - Robot #3 Şafağın Robotları_hocr.md_tokens.pkl
md/Diyanet Vakfı İslam Ansiklopedisi - 35 (Resuliler)_hocr.md_tokens.pkl
md/Ceza Kolonisinde - Franz Kafka_hocr.md_tokens.pkl
md/Lev Nikolayeviç Tolstoy - Hayatın Anlamı_hocr.